<a href="https://colab.research.google.com/github/riyalguha/Poem_generator/blob/main/poem_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import warnings

warnings.filterwarnings("ignore")
os.environ["TF__CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import time

import tensorflow as tf
import numpy as np

## Download Dataset

In [3]:
path_to_file = tf.keras.utils.get_file("shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt",)

In [4]:
text = open(path_to_file,"rb").read().decode(encoding="utf-8")
print(f"The length of text is {len(text)} characters")

The length of text is 1115394 characters


In [5]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
vocab = sorted(set(text))
print(f"{len(vocab)} unique characters")

65 unique characters


## Processing The Text

### Vectorizing the text

In [7]:
example_texts = ["abcdefg", "xyz"]

# TODO 1
chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [8]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [10]:
chars_from_ids =  tf.keras.layers.StringLookup(
    vocabulary = ids_from_chars.get_vocabulary(),invert = True,mask_token = None
)

In [11]:
chars = chars_from_ids(ids)

In [12]:
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [13]:
tf.strings.reduce_join(chars,axis = -1).numpy()## Joins tensors row-wise

array([b'abcdefg', b'xyz'], dtype=object)

In [14]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids),axis = -1)

## Prediction Task

In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(text,input_encoding = "UTF-8"))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [16]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [17]:
for ids in ids_dataset.take(18):
  print(chars_from_ids(ids).numpy().decode("utf-8"))


F
i
r
s
t
 
C
i
t
i
z
e
n
:


B
e
f


In [18]:
seq_length = 100
examples_per_epochs = len(text) // (seq_length+1)

In [19]:
sequences = ids_dataset.batch(seq_length+1,drop_remainder = True)

for seq in sequences.take(1):
  print(chars_from_ids(seq).numpy())

[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' ']


In [20]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


### Now we need to divide each sequence into (input,label) pairs where input is the current character and label is the next character

In [21]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [22]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [23]:
dataset = sequences.map(split_input_target)

In [24]:
for input_exp,target_exp in dataset.take(1):
  print("INPUT: ",text_from_ids(input_exp).numpy(),len(input_exp.numpy()))
  print("TARGET: ",text_from_ids(target_exp).numpy(),len(target_exp.numpy()))

INPUT:  b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou' 100
TARGET:  b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou ' 100


## Creating Training Batches

In [25]:
#Batch size
batch_size = 64

buffer_size = 10000
dataset = (
    dataset.shuffle(buffer_size)
    .batch(batch_size,drop_remainder = True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Building the model


In [26]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [27]:
# class MyModel(tf.keras.Model):
#   def __init__(self,vocab_size,embedding_dim,rnn_units):
#     super().__init__(self)
#     self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_dim)#generates embeddings vectors from the integer encoded vocabulary indices
#     self.gru1 = tf.keras.layers.GRU(rnn_units,return_sequences = True, return_state = True)#the RNN layer
#     self.gru2 = tf.keras.layers.GRU(rnn_units,return_sequences = True,return_state = True)
#     self.dense = tf.keras.layers.Dense(vocab_size)#generates the prob for next char

#   def call(self,inputs,states = None, return_state = False, training = False):
#     x = self.embedding(inputs,training = training)
#     if states is None:
#       states1 = self.gru1.get_initial_state(x)
#       states2 = self.gru2.get_initial_state(x)
#     else
#       states1,states2 = states
#     x,states1 = self.gru1(x,initial_state = states1, training = training)
#     x.states2 = self.gru2(x,initial_state = states2,training = training)
#     x = self.dense(x,training = training)

#     if return_state:
#       return x,[states1,states2]
#     else:
#       return x

In [28]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        # TODO - Create an embedding layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # TODO - Create a GRU layer
        self.gru = tf.keras.layers.GRU(
            rnn_units, return_sequences=True, return_state=True
        )
        # TODO - Finally connect it with a dense layer
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        # since we are training a text generation model,
        # we use the previous state, in training. If there is no state,
        # then we initialize the state
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [29]:
model = MyModel(vocab_size = len(ids_from_chars.get_vocabulary()),
                embedding_dim = embedding_dim,
                rnn_units = rnn_units
                )

## Try The Model

In [30]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape,
        "# (batch_size,sequence_length,vocab_size)")

(64, 100, 66) # (batch_size,sequence_length,vocab_size)


In [31]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
sampled_indices = tf.random.categorical(
      example_batch_predictions[0],num_samples = 1
)
sampled_indices = tf.squeeze(sampled_indices,axis = -1).numpy()

In [33]:
sampled_indices

array([18, 20, 56, 47, 25, 65, 38, 13, 19, 51, 18, 55, 26, 61, 17, 34, 34,
       17, 20, 45, 63, 44,  5, 24, 10,  2, 45, 10, 25, 17, 35, 53, 13, 12,
       12, 55, 34, 51,  8,  2, 31,  7, 53,  2,  5, 58, 62, 64, 61, 16,  5,
       40, 16, 47, 52, 50,  5, 19, 26, 47, 13, 61, 37, 29, 45, 15, 33, 55,
       24, 65, 28, 26, 26,  8, 54, 61, 50, 45,  6, 39, 54, 36, 17, 41, 46,
       42, 17, 19, 56,  0, 32, 64, 32, 32,  7, 11, 25, 64, 52, 17])

In [34]:
print("Input: \n",text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions: \n",text_from_ids(sampled_indices).numpy())

Input: 
 b" speed, is gone.\n\nLEONTES:\nHow! gone!\n\nServant:\nIs dead.\n\nLEONTES:\nApollo's angry; and the heavens t"

Next Char Predictions: 
 b"EGqhLzY?FlEpMvDUUDGfxe&K3 f3LDVn?;;pUl- R,n &swyvC&aChmk&FMh?vXPfBTpKzOMM-ovkf'ZoWDbgcDFq[UNK]SySS,:LymD"


## Train The Model


In [35]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits = True)

In [36]:
example_batch_mean_loss = loss(target_example_batch,example_batch_predictions)
print("Prediction Shape: ",
      example_batch_predictions.shape,
      "# (batch_size,sequence_length,vocab_size)")
print("Mean Loss   :",example_batch_mean_loss)

Prediction Shape:  (64, 100, 66) # (batch_size,sequence_length,vocab_size)
Mean Loss   : tf.Tensor(4.1892514, shape=(), dtype=float32)


In [37]:
tf.exp(example_batch_mean_loss).numpy()

65.97339

In [38]:
model.compile(optimizer = 'adam',loss = loss)

## Configure Checkpoints

In [39]:
checkpoint_dir = "./training_checkpoints"
checkpoint_prefix = os.path.join(checkpoint_dir,"ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix, save_weights_only = True
)

In [40]:
EPOCHS = 30

In [41]:
history = model.fit(dataset, epochs = EPOCHS, callbacks = [checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 21s 66ms/step - loss: 2.7179
Epoch 2/30
172/172 [==============================] - 12s 59ms/step - loss: 1.9850
Epoch 3/30
172/172 [==============================] - 12s 61ms/step - loss: 1.7102
Epoch 4/30
172/172 [==============================] - 12s 62ms/step - loss: 1.5497
Epoch 5/30
172/172 [==============================] - 12s 62ms/step - loss: 1.4514
Epoch 6/30
172/172 [==============================] - 12s 62ms/step - loss: 1.3836
Epoch 7/30
172/172 [==============================] - 12s 62ms/step - loss: 1.3308
Epoch 8/30
172/172 [==============================] - 13s 62ms/step - loss: 1.2865
Epoch 9/30
172/172 [==============================] - 13s 62ms/step - loss: 1.2454
Epoch 10/30
172/172 [==============================] - 13s 62ms/step - loss: 1.2060
Epoch 11/30
172/172 [==============================] - 12s 62ms/step - loss: 1.1664
Epoch 12/30
172/172 [==============================] - 12s 63ms/step - loss: 1.1252
E

In [42]:
class OneStep(tf.keras.Model):
  def __init__(self,model,chars_from_ids,ids_from_chars,temperature = 1.0):
    super().__init__()
    self.temperature = temperature
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars
    self.model = model

    # Create a mask to prevent "[UNK]" from being generated
    skip_ids = self.ids_from_chars(["UNK"])[:,None] # ADDs a new axis to skip_ids
    sparse_mask = tf.SparseTensor(
        values = [-float("inf")]*len(skip_ids),
        indices = skip_ids,
        dense_shape = [len(ids_from_chars.get_vocabulary())]
    )
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self,inputs,states = None):
    input_chars = tf.strings.unicode_split(inputs,"UTF-8")
    input_ids = self.ids_from_chars(input_chars).to_tensor()
    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs = input_ids,
                                          states = states,
                                          return_state = True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:,-1,:]
    predicted_logits = predicted_logits / self.temperature
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits,num_samples = 1)
    predicted_ids = tf.squeeze(predicted_ids, axis = -1)

    predicted_chars = self.chars_from_ids(predicted_ids)

    return predicted_chars, states




In [43]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [44]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(
      next_char,states = states
  )
  result.append(next_char)
result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'),"\n\n"+"_"*80)
print("\nRun Time" , end - start)


ROMEO:
Soft! with my sword, I'll say anon.

SICINIUS:
We will bestreme good scoff? so thou!

LUCIO:
Do not break his native wrongs
To speak it into so deal out, if you live,
To bitter where she purish me
For being a little with mildny.
But now the Duke of York is set on victory!
Is't not the truth make me wife of it?
Who hath a stirring drum? For now are dump,
Lest to the queen's allied: shall I go see?

LARTIUS:
Dishonourable thou, joy in the thing, once more
To-notice is leason.
Hark you, like a cordial party be your king;
But O, the horr perfidment should speak for requital
powers call not so.
Orge is the world--
The father 'scaped Hisood to seem to kisl
Tybalt waked from your exeliash,
That broke her brothers harded my son.

JOHN OF GAUNT:
To sun by great parknce! no fourteen young Dies for the
parter'st of those power: but 'tis done, gentle; earlish'd
Your dauly sea, and that you will hence
and do the friend: how could he stayd aloof,
To die, and go with me into men: advise
Here f

In [45]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, "\n\n" + "_" * 80)
print("\nRun time:", end - start)

tf.Tensor(
[b"ROMEO:\nA thousand time I'ld bid my peace by my bearing against his hateful.\n\nKING RICHARD III:\nWhat stayed you have not wander'd to me with you?\nHew does hence, do not proud me king;\nFor God's sake, Richard, will not hear these sad tisle,\nNay doth the pale clean-pinch'd than how.\nI would take from of this is the rackous elects;\nThat I, Somerver no dearer to hold you;\nWhat answer many suits, and sword,\nBut that thou sog or beauty dames that warm'd,\nThen mother, wretched months happier and her grace!\n\nGLOUCESTER:\nHow do you that friar?\n\nKING RICHARD III:\nWhat is it?\n\nGLOUCESTER:\nAlas, alas! she's not so. King Richard like a dead!\nA child servant serving and her reputation that\nI should do, and towards us all arrived,\nAnd Warwick be spoken of my bragglent next;\nAnd therefore, if you gave me to thine, she beg his\ndeath: there is no more remember and process.\nThis, do you perset!\n\nROMEO:\nAy, nurse; what he dies hour?\n\nMENENIUS:\nI thought, divid

In [46]:
tf.saved_model.save(one_step_model, "one_step")